In [13]:
# Manipule data
import pandas as pd


# Visualize data
import seaborn as sns 
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

# 1.2 Load DataSets

## 1.2.1 SEEG Brasil

### Tabela Geral de Dados Nacionais e Estaduais 

In [2]:
url = 'https://seeg.eco.br/wp-content/uploads/2024/02/SEEG11.1-DADOS-NACIONAIS.xlsx'

brasil_df = pd.read_excel(url, sheet_name=1)

In [1]:
brasil_df.shape

NameError: name 'brasil_df' is not defined